<a href="https://colab.research.google.com/github/Pranav-cosmos/First/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

primary = pd.read_csv("/content/train (1).csv")
dataset = Dataset.from_pandas(primary)
dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = dataset["train"]
test_dataset = dataset["test"]
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["message"], padding="max_length", truncation=True, max_length=32)

train_dataset=train_dataset.map(tokenize,batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

train_dataset = train_dataset.rename_column("label", "labels")
test_dataset = test_dataset.rename_column("label", "labels")
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/64000 [00:00<?, ? examples/s]

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
)
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

trainer.train()

Step,Training Loss
50,0.668600
100,0.609900
150,0.558900
200,0.522900
250,0.567400
300,0.540500
350,0.528600
400,0.519600
450,0.536300
500,0.493000


Step,Training Loss
50,0.668600
100,0.609900
150,0.558900
200,0.522900
250,0.567400
300,0.540500
350,0.528600
400,0.519600
450,0.536300
500,0.493000


NameError: name 'accuracy_score' is not defined

In [ ]:
# 11. Evaluate
from sklearn.metrics import accuracy_score, f1_score
results = trainer.evaluate()
print("Final Results:", results)

Final Results: {'eval_loss': 0.5027872920036316, 'eval_accuracy': 0.7818125, 'eval_f1': 0.7785881905245132, 'eval_runtime': 35.9601, 'eval_samples_per_second': 444.937, 'eval_steps_per_second': 27.809, 'epoch': 2.0}


In [ ]:
df = pd.read_csv("/content/test.csv")
print(df.head())
from datasets import Dataset
from transformers import BertTokenizer

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# HuggingFace Dataset from pandas
dataset = Dataset.from_pandas(df)

# Define tokenize function
def tokenize(batch):
    return tokenizer(batch["message"], padding="max_length", truncation=True)

# Apply
dataset = dataset.map(tokenize, batched=True)

predictions = trainer.predict(dataset)
y_pred = predictions.predictions.argmax(-1)

# Add predictions back to DataFrame
df["prediction"] = y_pred
print(df.head())

       row_id                                            message
0  1693632054                                        Goodnight! 
1  1469560531                          Watching Good Luck Chuck 
2  1984100549  shower then bed. maths in the morning. night t...
3  2176726866                     #trackle contest starts today 
4  2072130814  12-9 day tomorrow - on the bright side, I saw ...


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

       row_id                                            message  prediction
0  1693632054                                        Goodnight!            1
1  1469560531                          Watching Good Luck Chuck            1
2  1984100549  shower then bed. maths in the morning. night t...           0
3  2176726866                     #trackle contest starts today            1
4  2072130814  12-9 day tomorrow - on the bright side, I saw ...           0


In [ ]:
df = df[["row_id", "prediction"]]
print(df.head())
df.info()

       row_id  prediction
0  1693632054           1
1  1469560531           1
2  1984100549           0
3  2176726866           1
4  2072130814           0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   row_id      10000 non-null  int64
 1   prediction  10000 non-null  int64
dtypes: int64(2)
memory usage: 156.4 KB


In [ ]:
df.to_csv("predictions.csv", index=False)
print("Predictions saved to predictions.csv")

Predictions saved to predictions.csv


In [ ]:
from google.colab import files
files.download("predictions.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>